# TP1

In [17]:
import pandas as pd

def loadData(file):
    # Charger les données CSV en spécifiant les noms de colonnes
    data = pd.read_csv(file, index_col=3,  
                       names=['gender', 'genderLooking', 'age', 'name', 'counts_details', 'counts_pictures', 
                              'counts_profileVisits', 'counts_kisses', 'counts_fans', 'counts_g', 
                              'flirtInterests_chat', 'flirtInterests_friends', 'flirtInterests_date', 
                              'country', 'city', 'location', 'distance', 'isFlirtstar', 'isHighlighted', 
                              'isInfluencer', 'isMobile', 'isNew', 'isOnline', 'isVip', 'lang_count', 
                              'lang_fr', 'lang_en', 'lang_de', 'lang_it', 'lang_es', 'lang_pt', 'verified', 
                              'shareProfileEnabled', 'lastOnlineDate', 'lastOnlineTime', 'birthd', 'crypt', 
                              'freetext', 'whazzup', 'userId', 'pictureId', 'isSystemProfile'])
    
   
    # CONVERSION DE TYPES DE DONNEES
   
    # Convertir les colonnes 'counts_profileVisits' et 'counts_kisses' en numériques
    data['counts_profileVisits'] = pd.to_numeric(data['counts_profileVisits'], errors='coerce')
    data['counts_kisses'] = pd.to_numeric(data['counts_kisses'], errors='coerce')
                
    return data

# Charger les données depuis le fichier CSV
dataset = loadData("lovoo_v3_users_api-results.csv")

# Colonnes à supprimer du DataFrame
colonnes_a_supprimer = ['age', 'gender', 'genderLooking', 'counts_details', 'counts_pictures', 'counts_fans', 'counts_g',
                        'flirtInterests_chat', 'flirtInterests_friends', 'flirtInterests_date', 
                        'country', 'city', 'location', 'distance', 'isFlirtstar', 'isHighlighted',
                        'isInfluencer', 'isMobile', 'isNew', 'isOnline', 'isVip', 'lang_count',
                        'lang_fr', 'lang_en', 'lang_de', 'lang_it', 'lang_es', 'lang_pt', 'verified',
                        'shareProfileEnabled', 'lastOnlineDate', 'birthd', 'crypt',
                        'freetext', 'whazzup', 'userId', 'isSystemProfile']

# Supprimer les colonnes spécifiées du DataFrame
dataset = dataset.drop(colonnes_a_supprimer, axis=1)


# SUPPRESSION DES DONNEES DUPLIQUEES 

# Filtrer les enregistrements qui ne sont pas dupliqués ou où pictureId est blank
dataset_unique = dataset[~dataset['pictureId'].duplicated() | dataset['pictureId'].isnull()]

# Affectation des résultats uniques au jeu de données original
dataset = dataset_unique.copy()

# Création de nouvelles caractéristiques : 

# Calculer kisses_per_visit et arrondir à deux décimales
dataset['taux_aime'] = (dataset['counts_kisses'] / dataset['counts_profileVisits'] * 100).round(2)

# DISCRÉTISATION

# Discrétisation de la variable taux_aime dans une nouvelle colonne 'aime_5_pct', selon les conditions spécifiées
dataset['aime_5_pct'] = pd.cut(dataset['taux_aime'], bins=[-float('inf'), 5, float('inf')], labels=['moins_5_pct_aime', 'plus_5_pct_aime'], right=False)

# Discrétisation de la variable counts_profileVisits dans une nouvelle colonne '10_visites' selon les conditions spécifiées
dataset['10_visites'] = dataset.apply(lambda row: 'plus_10_visites' if row['counts_profileVisits'] >= 10 else 'moins_10_visites', axis=1)
   
# COMPTE POPULAIRE
   
# Créer la nouvelle colonne 'compte_populaire' basée sur les conditions spécifiées
dataset['compte_populaire'] = dataset.apply(lambda row: 'populaire' if row['aime_5_pct'] == 'plus_5_pct_aime' and row['10_visites'] == 'plus_10_visites' else 'non_populaire', axis=1)
    

# NORMALISATION

# Normaliser la colonne counts_profileVisits
dataset['norm_visites'] = (dataset['counts_profileVisits'] / dataset['counts_profileVisits'].max()).round(3)

# Normaliser la colonne counts_kisses
dataset['norm_bisous'] = (dataset['counts_kisses'] / dataset['counts_kisses'].max()).round(3)

# IDENTIFICATION DES INCOHÉRENCES

# Filtrer les lignes où counts_bisous est inférieur ou égal à counts_visitesProfil
dataset = dataset[dataset['counts_kisses'] <= dataset['counts_profileVisits']]

# Afficher les premières lignes du DataFrame avec les nouvelles colonnes ajoutées
print(dataset.iloc[:5])

print("\n")

# TOTAUX

# Compter combien de fois "populaire" apparaît dans la colonne compte_populaire
count_populaire = (dataset['compte_populaire'] == 'populaire').sum()

# Afficher le nombre de fois où "populaire" apparaît dans la colonne compte_populaire
print(f'Nombre de valeurs "populaire": {count_populaire}')

# Compter combien de fois "plus_10_visites" apparaît dans la colonne 10_visites
count_plus_10_visites = (dataset['10_visites'] == 'plus_10_visites').sum()

print(f'Nombre de "plus_10_visites": {count_plus_10_visites}')

# Compter combien de fois "plus_5_pct_aime" apparaît dans la colonne aime_5_pct
count_plus_5_pct_aime = (dataset['aime_5_pct'] == 'plus_5_pct_aime').sum()

print(f'Nombre de "plus_5_pct_aime": {count_plus_5_pct_aime}')




              counts_profileVisits  counts_kisses lastOnlineTime  \
name                                                               
daeni                       8279.0          239.0     1429994606   
italiana 92                  663.0           13.0     1430039975   
Lauraaa                     1369.0           88.0     1428330247   
Qqkwmdowlo                 22187.0         1015.0     1428405661   
schaessie {3               35262.0         1413.0     1428330320   

                             pictureId  taux_aime        aime_5_pct  \
name                                                                  
daeni         55303fc3160ba0eb728b4575       2.89  moins_5_pct_aime   
italiana 92   552e7b61c66da10d1e8b4c82       1.96  moins_5_pct_aime   
Lauraaa       54a584ecc56da128638b4674       6.43   plus_5_pct_aime   
Qqkwmdowlo    54c92738076ea1b5338b4735       4.57  moins_5_pct_aime   
schaessie {3  54e1a6f6c76da135748b4a3a       4.01  moins_5_pct_aime   

                   10_vis